In [ ]:
class Friction:

    with open('system_new.txt', 'r') as txt:
        system = '\n'.join(txt.readlines())

    def prompt(x):
        p = f'The example dialogue is provided next.\n\n{x["context"]}\n\n\n'
        try:
            p += f'The response is provided below.\n\nResponse: {x["response"]}\n\n\n'
        except KeyError:
            p += f'The response is provided below.\n\nResponse: \n\n\n'
        p += 'What friction category if any?'
        return p

f = Friction()

In [ ]:
import datasets

dataset = datasets.load_dataset('pfb30/multi_woz_v22', split='train')


In [ ]:
d = dataset[0]
turns = d['turns']
num_turns = len(turns['utterance'])
for i in range(num_turns):
    u = turns['utterance'][i]
    a = turns['dialogue_acts'][i]
    speaker_id = turns['speaker'][i]
    if speaker_id == 1:
        print(f"\nAssistant: {u}\n\t(Dialogue acts: {','.join(a['dialog_act']['act_type'])})")
    else:
        print(f"\nUser: {u}")

In [ ]:
all_acts = []
for d in dataset:
    for a in d['turns']['dialogue_acts']:
        all_acts.extend(a['dialog_act']['act_type'])
print(len(all_acts))
print(len(set(all_acts)))
print(set(all_acts))

In [ ]:
all_services = sum([d['services'] for d in dataset], [])
print(set(all_services))

In [ ]:
from tqdm import tqdm
all_selected_instances = []
for act in tqdm(set(all_acts)):
    if act.startswith('general'):
        continue
    selected_instances = []
    for d in dataset:
        turns = d['turns']
        num_turns = len(turns['utterance'])
        for i in range(num_turns):
            u = turns['utterance'][i]
            acts = turns['dialogue_acts'][i]['dialog_act']['act_type']
            speaker_id = turns['speaker'][i]
            if speaker_id == 1 and acts == [act]:
                context = ""
                for prev_i in range(i):
                    if turns['speaker'][prev_i] == 0:
                        context += f"User: {turns['utterance'][prev_i]}\n"
                    else:
                        context += f"Assistant: {turns['utterance'][prev_i]}\n"
                new_d = {
                    "context": context,
                    "response": f"Assistant: {u}",
                    "dialogue_acts": acts,
                }
                selected_instances.append(new_d)
                if len(selected_instances) == 10:
                    break
        if len(selected_instances) == 10:
                    break

    #print(len(selected_instances))
    all_selected_instances.extend(selected_instances)
print(f"Selected {len(all_selected_instances)} instances")

In [ ]:
acttypes = ['book', 'inform', 'nobook', 'nooffer', 'offerbook', 'offerbooked', 'recommend', 'request', 'select']
from tqdm import tqdm
all_selected_instances = []
for act in tqdm(acttypes):
    if act.startswith('general'):
        continue
    selected_instances = []
    for d in dataset:
        turns = d['turns']
        num_turns = len(turns['utterance'])
        for i in range(num_turns):
            u = turns['utterance'][i]
            acts = turns['dialogue_acts'][i]['dialog_act']['act_type']
            speaker_id = turns['speaker'][i]
            #print(acts, act)
            #break
            if speaker_id == 1 and len(acts) == 1 and acts[0].lower().endswith(act):
                context = ""
                for prev_i in range(i):
                    if turns['speaker'][prev_i] == 0:
                        context += f"User: {turns['utterance'][prev_i]}\n"
                    else:
                        context += f"Assistant: {turns['utterance'][prev_i]}\n"
                new_d = {
                    "context": context,
                    "response": f"Assistant: {u}",
                    "dialogue_acts": acts,
                }
                selected_instances.append(new_d)
                if len(selected_instances) == 50:
                    break
        if len(selected_instances) == 50:
                    break

    #print(len(selected_instances))
    all_selected_instances.extend(selected_instances)
print(f"Selected {len(all_selected_instances)} instances")

In [ ]:
all_selected_acts = sum([d['dialogue_acts'] for d in all_selected_instances], [])
all_selected_act_types = [x.split('-')[-1].lower() for x in all_selected_acts]
print(len(all_selected_acts))
from collections import Counter
print(Counter(all_selected_act_types))


In [ ]:
import random
from openai_utils import openai_caller

idx = random.choice(range(len(all_selected_instances)))
d = all_selected_instances[idx]
print(f"Context: {d['context']}")
print(f"Response: {d['response']}")
print(f"Dialog acts: {d['dialogue_acts']}")

prompt = Friction.system + Friction.prompt(d)
messages = [{'role': 'system', 'content': prompt}]

gpt_response = openai_caller(messages, max_new_tokens=256, model='gpt4o')

In [ ]:
gpt_response

In [ ]:
openai_caller.compute_cost()

In [ ]:
openai_caller.reset_tokens_used()
annotated_data = []

for d in tqdm(all_selected_instances):
        prompt = Friction.system + Friction.prompt(d)
        messages = [{'role': 'system', 'content': prompt}]
        gpt_response = openai_caller(messages, max_new_tokens=256, model='gpt4o')
        friction_anno = gpt_response.split('ANSWER = ')[-1]
        new_d = d.copy()
        new_d['friction_anno'] = friction_anno
        new_d['gpt_response'] = gpt_response
        annotated_data.append(new_d)
print(f"Annotation cost: ${openai_caller.compute_cost():.4f}")

In [ ]:
from collections import Counter
annotations = [x['friction_anno'] for x in annotated_data]
print(Counter(annotations))

In [ ]:
while True:
    x = random.choice(annotated_data)
    if "NoBook" in x['dialogue_acts'][0]:
        break
print(f"Context: {x['context']}")
print(f"Response: {x['response']}")
print(f"Dialog acts: {x['dialogue_acts']}")
print(f"Friction annotation: {x['friction_anno']}")


In [ ]:
acttype_frictionanno_counter = Counter()
for d in annotated_data:
    for a in d['dialogue_acts']:
        acttype = a.split('-')[-1].lower()
        if len(d['friction_anno']) > 20:
            continue
        acttype_frictionanno_counter[(acttype, d['friction_anno'])] += 1

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

rows = set([x[0] for x in acttype_frictionanno_counter.keys()])
cols = set([x[1] for x in acttype_frictionanno_counter.keys()])
data = [[acttype_frictionanno_counter[(r, c)]/sum([acttype_frictionanno_counter[(r, c2)] for c2 in cols]) for c in cols] for r in rows]
df = pd.DataFrame(data)#, index=rows, columns=cols)
plt.figure(figsize=(8, 8), dpi=150)
sns.heatmap(df, annot=True, fmt='.2f', cmap='viridis')
plt.xlabel('Friction Category', fontsize=14)
plt.ylabel('Dialog Act type', fontsize=14)
plt.xticks(ticks=[x+0.5 for x in range(len(cols))], labels=cols, rotation=45, fontsize=12)
plt.yticks(ticks=[x+0.5 for x in range(len(rows))], labels=rows, rotation=0, fontsize=12)
plt.title('Fraction of instances in each MultiWoZ Dialog Act type \nclassified under different Friction categories', fontsize=14)
plt.show()

In [ ]:
import json
json.dump(annotated_data, open(f'data_new_prompt/multiwoz_friction_anno-{len(annotated_data)}instances.json', 'w'), indent=2)

In [ ]:
rows
for r in rows:
    print(r, sum([acttype_frictionanno_counter[(r, c)] for c in cols]))

In [7]:
import json

annotations = json.load(open('data_new_prompt/multiwoz_friction_anno-450instances.json'))
for x in annotations:
    x['dialog_act_type'] = x['dialogue_acts'][0].split('-')[-1].lower()

In [8]:
from collections import defaultdict

das = set([x['dialog_act_type'] for x in annotations])
da2annos = {da: [x for x in annotations if x['dialog_act_type'] == da] for da in das}
fcs = set([x['friction_anno'] for x in annotations])
fc2annos = {fc: [x for x in annotations if x['friction_anno'] == fc] for fc in fcs}
dafc2annos = {(da, fc): [x for x in annotations if x['dialog_act_type'] == da and x['friction_anno'] == fc] for da in das for fc in fcs}

In [13]:
import random
#x = random.choice(da2annos["MiscOther"])
#x = random.choice(fc2annos["Probing"])
x = random.choice(dafc2annos[('select', 'Probing')])

print(x['context'])

print("RESPONSE TO ANNOTATE:", x['response'])

print("\nANNOTATION:", x['friction_anno'])
print("GPT REASONING:", x['gpt_response'])


User: I need to find a swimming pool in the north.
Assistant: There are 2 swimming pools in the north, jesus green outdoor pool and kings hedges learner pool. Which one would you like more information on?
User: Kings Hedges sounds good, what postcode are they in? And do they have an entrance fee?
Assistant: Kings Hedges Learner Pool is in postcode cb42xh. Their entrance fee is a bit of a mystery, however, I don't have any firm numbers. Can I help with anything else?
User: Yes, I need some information about a restaurant called pizza express.
Assistant: Certainly! What information specifically are you looking for?
User: I'd like to know if I can book a reservation for 7 people at 18:00 monday.

RESPONSE TO ANNOTATE: Assistant: Were you looking for a booking at Pizza Express or Pizza Express Fen Ditton? They are both in the centre of town.

ANNOTATION: Probing
GPT REASONING: The response "Assistant: Were you looking for a booking at Pizza Express or Pizza Express Fen Ditton? They are both